In [1]:
%matplotlib inline
!pip install gurobipy

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import gurobipy as gp
from gurobipy import GRB
from time import time

## Question 1:

In [2]:
def read_data():
    data = pd.read_csv('constants.csv')
    P = data['value'][0]
    R = data['value'][1]
    n = data['value'][2]
    return (P,R,n)

In [3]:
def instance_data(number_of_instance):
    instance = pd.read_csv('instance_'+str(number_of_instance)+'.csv')
    m = instance.shape[0]
    p = []
    r = []
    for p_i in instance['p_i']:
        p.append(p_i)
    for r_i in instance['r_i']:
        r.append(r_i)
    return (m,p,r)

In [4]:
def Model( number_of_instance , m , p , r , P , R , n ):
    global model, x_vars , y_vars , run_time
    model = gp.Model("q1_instance_"+str(number_of_instance))
    model.Params.Presolve = 0
    x_vars = {}
    y_vars = {}
    start = time()
    for i in range(n):
        for j in range(m):
            x_vars[i,j] = 0 
    for i in range(m):
        y_vars [i] = 0
    constructVars() 
    contructConstrs()
    constructObj() 
    model.write('test.lp')
    model.optimize()

In [5]:
def constructVars(): 
    global n , m , model, x_vars ,y_vars
    for i in range( m ): 
        for j in range( n ): 
            var = model.addVar( vtype = GRB.BINARY , name = "x_" + str( i+1 ) + "_" + str( j+1 ) ) 
            x_vars[ i , j ] = var 
    model.update()
    for j in range( n ):
        var_2 = model.addVar( vtype = GRB.BINARY , name = "y_" + str( j+1 )) 
        y_vars[ j ] = var_2
    model.update()

In [6]:
def contructConstrs(): 
    global n , m , model , x_vars , y_vars , R, P
    for i in range( m ): 
        constExpr_0 = gp.LinExpr() 
        for j in range( n ): 
            constExpr_0 += 1.0 * x_vars[ i,j ] 
        model.addConstr( lhs = constExpr_0 , sense = GRB.EQUAL , rhs = 1 ) 

    for i in range( n ):
        constExpr_1_1 = gp.LinExpr()
        constExpr_1_2 = gp.LinExpr()
        for j in range( m ):
            constExpr_1_1 = 1.0 * x_vars[i,j]
            constExpr_1_2 = 1.0 * y_vars[j]
            model.addConstr(lhs= constExpr_1_1, sense = GRB.LESS_EQUAL , rhs = constExpr_1_2)
            
    for j in range( n ):
        constExpr_2 = gp.LinExpr()
        rhs_exp = gp.LinExpr()
        for i in range( m ): 
            constExpr_2 += p[j] * x_vars[ i,j ]
            rhs_exp = P * y_vars[j] 
        model.addLConstr( name = 'memory capacity constraints'+str(j+1),lhs = constExpr_2 , sense = GRB.LESS_EQUAL , rhs = rhs_exp  ) 
    model.update()
    
    for j in range( n ):
        constExpr_3 = gp.LinExpr()
        rhs_exp_3 = gp.LinExpr()
        for i in range( m ): 
            constExpr_3 += r[j] * x_vars[ i,j ]
            rhs_exp_3 = R * y_vars[j] 
        model.addLConstr( name = 'time capacity constraints'+str(j+1),lhs = constExpr_3 , sense = GRB.LESS_EQUAL , rhs = rhs_exp_3  ) 
    model.update()

In [7]:
def constructObj(): 
    global m , n , model , x_vars , y_vars
    objExpr = gp.LinExpr()
    for j in range( n ):
        objExpr += y_vars[j]
    model.setObjective( objExpr , GRB.MINIMIZE ) 
    model.update()

In [8]:
def processing_result(): #source: Gurobi website (https://www.gurobi.com/documentation/9.5/examples/mip2_py.html)
    # Read and solve model

    if model.IsMIP == 0:
        print('Model is not a MIP')
        sys.exit(0)

    model.optimize()

    if model.Status == GRB.OPTIMAL:
        print('Optimal objective: %g' % model.ObjVal)
    elif model.Status == GRB.INF_OR_UNBD:
        print('Model is infeasible or unbounded')
        sys.exit(0)
    elif model.Status == GRB.INFEASIBLE:
        print('Model is infeasible')
        sys.exit(0)
    elif model.Status == GRB.UNBOUNDED:
        print('Model is unbounded')
        sys.exit(0)
    else:
        print('Optimization ended with status %d' % model.Status)
        sys.exit(0)

    # Iterate over the solutions and compute the objectives
    model.Params.OutputFlag = 0
    print('')
    for k in range(model.SolCount):
        model.Params.SolutionNumber = k
        print('Solution %d has objective %g' % (k, model.PoolObjVal))
    print('')
    model.Params.OutputFlag = 1

    fixed = model.fixed()
    fixed.Params.Presolve = 0
    fixed.optimize()

    if fixed.Status != GRB.OPTIMAL:
        print("Error: fixed model isn't optimal")
        sys.exit(1)

    diff = model.ObjVal - fixed.ObjVal

    if abs(diff) > 1e-6 * (1.0 + abs(model.ObjVal)):
        print('Error: objective values are different')
        sys.exit(1)

In [9]:
for i in range(10):
    number_of_instance = i
    m , p , r = instance_data( number_of_instance )
    P,R,n = read_data()
    print('\n instnce '+str(number_of_instance)+'\n--------------------')
    Model( number_of_instance , m , p , r , P , R , n)
    for k in range(model.SolCount):
        model.Params.SolutionNumber = k
        print('Solution %d has objective %g' % (k, model.PoolObjVal))
    print('\n----------------------------------------------------------------\n\n')


 instnce 0
--------------------
Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-19
Set parameter Presolve to value 0
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 270 rows, 240 columns and 1155 nonzeros
Model fingerprint: 0xb35a50b5
Variable types: 0 continuous, 240 integer (240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 11.0000000
Variable types: 0 continuous, 240 integer (240 binary)

Root relaxation: objective 5.508200e+00, 325 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.50820    0   12   11.00000    5.5

<ipython-input-4-a4d1628fcced>:14: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  contructConstrs()


Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 270 rows, 240 columns and 1155 nonzeros
Model fingerprint: 0xda80f289
Variable types: 0 continuous, 240 integer (240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 10.0000000
Variable types: 0 continuous, 240 integer (240 binary)

Root relaxation: objective 4.679127e+00, 365 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.67913    0   11   10.00000    4.67913  53.2%     -    0s
H    0     0                       7.0000000    4.67913  33.2%     -    0s
H    0     0                       6.0000000    4.67913  22

In [10]:
def result(): 
    # Function for printing values of nonzero variables
    model.optimize()
    fixed = model.fixed()
    fixed.Params.Presolve = 0
    fixed.optimize()
    for v in fixed.getVars():
        if v.X != 0:
            i = 0
            answer = ''
            if v.VarName[0] != 'y':
                if len(v.VarName) == 7:
                    if v.Varname[3] != '_':
                        i += 1
                        print('job',v.VarName[2:4],'is assigned to machine', v.VarName[5:len(v.VarName)])
                    else:
                        i += 1
                        print('job',v.VarName[2],'is assigned to machine', v.VarName[5:len(v.VarName)])
            
            
                if len(v.VarName) == 6:
                    if v.VarName[4] != '_':
                        i += 1
                        print('job',v.VarName[2],'is assigned to machine', v.VarName[4:len(v.VarName)])
                    else:
                        i += 1
                        print('job',v.VarName[2:4],'is assigned to machine', v.VarName[len(v.VarName)-1])
            
            
                if len(v.VarName) == 5:
                    i += 1
                    print('job',v.VarName[2],'is assigned to machine',v.VarName[len(v.VarName)-1])
    model.printAttr('ObjVal')

In [11]:
result() # result of the optimization

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 270 rows, 240 columns and 1155 nonzeros
Model fingerprint: 0x4658e50c
Variable types: 0 continuous, 240 integer (240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Continuing optimization...


Cutting planes:
  Gomory: 2
  MIR: 3
  GUB cover: 1
  RLT: 3

Explored 1 nodes (423 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 5 11 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.000000000000e+00, best bound 5.000000000000e+00, gap 0.0000%
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 270 rows, 240 columns and 

## relaxation:

In [19]:
def Model_2( number_of_instance , m , p , r , P , R , n ):
    global model, x_vars , y_vars , run_time
    model_2 = gp.Model("q1_instance_"+str(number_of_instance))
    x_vars = {}
    y_vars = {}
    start = time()
    for i in range(n):
        for j in range(m):
            x_vars[i,j] = 0 
    for i in range(m):
        y_vars [i] = 0
    constructVars() 
    contructConstrs()
    constructObj() 
    model.write('test.lp')
    model.optimize()
    model.relax()

In [22]:
for i in range(10):
    number_of_instance = i
    m , p , r = instance_data( number_of_instance )
    P,R,n = read_data()
    print('\n instnce '+str(number_of_instance)+'\n--------------------')
    Model_2( number_of_instance , m , p , r , P , R , n)
    print((model.MIPGap))
    print('\n----------------------------------------------------------------\n\n')


 instnce 0
--------------------
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8910 rows, 7920 columns and 38115 nonzeros
Model fingerprint: 0xc3b415ba
Variable types: 0 continuous, 7920 integer (7920 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

MIP start from previous solve produced solution with objective 10 (0.01s)
MIP start from previous solve produced solution with objective 8 (0.02s)
MIP start from previous solve produced solution with objective 7 (0.02s)
Loaded MIP start from previous solve with objective 7

Variable types: 0 continuous, 7920 integer (7920 binary)

Root relaxation: objective 5.508200e+00, 4622 iterations, 0.04 seconds (0.05 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  O

<ipython-input-19-efbe00530e1c>:13: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  contructConstrs()



Cutting planes:
  Gomory: 6
  MIR: 1
  GUB cover: 6
  RLT: 5

Explored 1 nodes (8535 simplex iterations) in 0.20 seconds (0.16 work units)
Thread count was 8 (of 8 available processors)

Solution count 3: 7 8 10 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.000000000000e+00, best bound 7.000000000000e+00, gap 0.0000%
0.0

----------------------------------------------------------------



 instnce 1
--------------------
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 9180 rows, 8160 columns and 39270 nonzeros
Model fingerprint: 0x626cc115
Variable types: 0 continuous, 8160 integer (8160 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

MIP start from previous solve produced solution with objective 6 (0.01s)
Loaded MIP start from previous 